In [1]:
import sys
sys.path.append('../')
import tokamakTK

from tokamakTK import get_ECT_regression

import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import scipy as sp
import statsmodels.api as sm
import matplotlib.patches as mpatches

pd.set_option('display.max_columns', None)
plt.rc('font',family = 'serif')
plt.rc('axes', axisbelow=True)

path = "../data/"
fig_path = "../../../LATEX/Latex Images/"
plt.rcParams["mathtext.fontset"] = "cm"

[Notes based on: Sarazin, et al. 2019](https://iopscience.iop.org/article/10.1088/1741-4326/ab48a5/meta)

In [9]:
esqueleto = {
    "TAUTH":None, 
    "BETASTAR":None,        
    "RGEO":None,  
    "BT":None,    
    "NEL":None,   
    "TAV":None,  
    "IP":None,   
    "PLTH":None,
    "Q95":None,
    "KAREA":None,
    "EPS":None,
    "MEFF":None
}


# IPB98(y,2)
DEMO = {"TAUTH":4.6, # s
        "BETASTAR":2.2, # 2.2
        "RGEO":9.1,  # m # Some say 7.5
        "BT":5.6,    # T
        "NEL":8.7,   # 10^19
        "TAV":12.9,  # keV
        "IP":19.6,   # MA
        "PLTH":157.5,# MW
        "Q95":3.2,
        "KAREA":1.59,
        "EPS":3.1,
        "MEFF":6.27
       }

ITER = {"TAUTH":3.1, # s
        "BETASTAR":2.43,# 2.2
        "RGEO":6.2,  # m
        "BT":4.42,   # T
        "NEL":8.4,   # 10^19
        "TAV":10.7,  # keV
        "IP":12.5,   # MA
        "PLTH": 106, # MW
        "Q95":3,
        "KAREA":1.7,
        "EPS":3.1,
        "MEFF":2.554
       }

# NBI-LHW Database
EAST = {"TAUTH":0.05647, # s
        "BETASTAR":None,# 2.2
        "RGEO":1.86,  # m
        "BT":1.97,   # T
        "NEL":3.36,   # 10^19
        "TAV":None,  # keV
        "IP":0.42,   # MA
        "PLTH":2.26, # MW
        "Q95":5.06,
        "KAREA":1.51,
        "EPS":0.44/1.86,
        "MEFF": 2.5 # THIS IS MY GUESSSSSSSSSSSSSSSSSSSSSSSSSS
       }

ST40 = {
    "TAUTH":None, 
    "BETASTAR":None,        
    "RGEO":0.4,  
    "BT":3,    
    "NEL":None,   
    "TAV":None,  
    "IP":2,   
    "PLTH":None,
    "Q95":1.8,
    "KAREA":2.6,
    "EPS":0.2/0.4,
    "MEFF":None,
    "Greewald":0.8,
    "MDG_beta":0.9
    
}


MAST_U = {
    "TAUTH":5,    # Changed to 2 in 2020
    "BETASTAR":None,        
    "RGEO":0.85,  
    "BT":0.75,  # Changed to 0.6 in 2020  
    "NEL":None,   
    "TAV":None,  
    "IP":2, # Changed to 1 in 2020
    "PLTH":None,
    "Q95":None,
    "KAREA":None,
    "EPS":0.65/0.85,
    "MEFF":None
}

MASTU_ICTP = {
    "TAUTH":0.055,    # Changed to 2 in 2020
    "BETASTAR":3.7,        
    "RGEO":0.85,  
    "BT":0.75,  # Changed to 0.6 in 2020  
    "NEL":4.55,   
    "TAV":1.41,  
    "IP":0.53, # Changed to 1 in 2020
    "PLTH":None,
    "Q95":None, #B_θ = 0.44T
    "KAREA":2.01,
    "EPS":0.65/0.85,
    "MEFF":None
}

ST_CTF = {
    "TAUTH":None,
    "BETASTAR":None,        
    "RGEO":0.84,  
    "BT":2.5,    
    "NEL":None,   
    "TAV":None,  
    "IP":6.5,   
    "PLTH":None,
    "Q95":None,
    "KAREA":None,
    "EPS":None,
    "MEFF":None
}

In [3]:
# Obtained from Optimization

min_subset_ids_6357 = pd.read_csv(path+"R_ids_alpha_0.6357.csv")
min_subset_ids_9998 = pd.read_csv(path+"R_ids_alpha_0.9998.csv")
min_subset_ids_joe  = pd.read_csv(path+"deviation_id.csv")

DB2 = pd.read_csv(path+"DB2P8.csv")
DB5 = pd.read_csv(path+"SELDB5_SVD.csv", low_memory=False) 

# Setting ELMy Dataset
DB5 = DB5[DB5["PHASE"].isin(['HGELM', 'HSELM', 'HGELMH', 'HSELMH'])]

# Removing Spherical TOKAMAKS
#DB5 = DB5[~DB5["TOK"].isin(['START','MAST','NSTX'])]

# There is two shots from DB2P8 missing in DB5
missing_shots = DB2[~DB2.id.isin( DB5.id.values )].reset_index(drop=True)
DB5 = pd.concat([DB5, missing_shots], axis=0, ignore_index=True)

# Labeling shots that had great impact in decreasing alpha_R
DB5.insert(loc=2,column="label_6357",value=["Unaffected"]*len(DB5))
DB5.loc[(DB5[DB5.id.isin(min_subset_ids_6357.id)].index), "label_6357"] = "Decreasing"

DB5.insert(loc=2,column="label_9998",value=["Unaffected"]*len(DB5))
DB5.loc[(DB5[DB5.id.isin(min_subset_ids_9998.id)].index), "label_9998"] = "Decreasing"

DB5.insert(loc=2,column="label_joe",value=["Unaffected"]*len(DB5))
DB5.loc[(DB5[DB5.id.isin(min_subset_ids_joe.id)].index), "label_joe"] = "Decreasing"

DB5.insert(loc=2,column="Spherical",value=["False"]*len(DB5))
DB5.loc[(DB5[DB5.TOK.isin(['START','MAST','NSTX'])].index), "Spherical"] = "True"

DB5.insert(0, 'intercept', np.ones(len(DB5)))


print(
    "  Subset that decrease alpha-R to 0.6357\n--------\n" +
    f"{ round( (len(min_subset_ids_6357)/len(DB5))*100     ,2)  }% affected alpha_R\n" + 
    f"{ round( (1 - len(min_subset_ids_6357)/len(DB5))*100 ,2)  }% did not affect alpha_R" +
    "\n\n\n  Subset that decrease alpha-R to 0.9998\n--------\n" +
    f"{ round( (len(min_subset_ids_9998)/len(DB5))*100     ,2)  }% affected alpha_R\n" + 
    f"{ round( (1 - len(min_subset_ids_9998)/len(DB5))*100 ,2)  }% did not affect alpha_R"
    "\n\n\n  Subset given by Joseph Hall           \n--------\n" +
    f"{ round( (len(min_subset_ids_joe)/len(DB5))*100     ,2)  }% affected alpha_R\n" + 
    f"{ round( (1 - len(min_subset_ids_joe)/len(DB5))*100 ,2)  }% did not affect alpha_R"
)

  Subset that decrease alpha-R to 0.6357
--------
23.45% affected alpha_R
76.55% did not affect alpha_R


  Subset that decrease alpha-R to 0.9998
--------
9.88% affected alpha_R
90.12% did not affect alpha_R


  Subset given by Joseph Hall           
--------
18.46% affected alpha_R
81.54% did not affect alpha_R


In [4]:
ITER

{'TAUTH': 3.1,
 'BETASTAR': 2.43,
 'RGEO': 6.2,
 'BT': 4.42,
 'NEL': 8.4,
 'TAV': 10.7,
 'IP': 12.5,
 'PLTH': 106,
 'Q95': 3,
 'KAREA': 1.7,
 'EPS': 3.1,
 'MEFF': 2.554}

In [5]:
DB2prms = get_ECT_regression(DB2).params.to_dict()
DB2prms['const'] = np.exp(DB2prms['const'])

In [6]:
DB5prms = get_ECT_regression(DB5).params.to_dict()
DB5prms['const'] = np.exp(DB5prms['const'])

In [7]:
DDsprms = get_ECT_regression(DB5[DB5.id.isin(min_subset_ids_6357.id.values) | 
                                 DB5.id.isin(DB2.id.values)
                                ]).params.to_dict()
DDsprms['const'] = np.exp(DDsprms['const'])

In [8]:
UDsprms = get_ECT_regression(DB5[~DB5.id.isin(min_subset_ids_6357.id.values)]).params.to_dict()
UDsprms['const'] = np.exp(UDsprms['const'])

In [9]:
UDsprms

{'const': 0.07543874049968176,
 'IP': 0.7818564549785937,
 'BT': 0.24388542411978847,
 'NEL': 0.41145103200443656,
 'PLTH': -0.7532206371481888,
 'RGEO': 2.155963618372499,
 'KAREA': 0.5002957204679769,
 'EPS': 0.7989791985662811,
 'MEFF': 0.2189922745716312}

$$
    \tau_{E,th} = \alpha_0\cdot I_P^{\alpha_I}\cdot B^{\alpha_B}_t\cdot \bar{n}^{\alpha_n}_e  \cdot P^{\alpha_P}_{l,th}\cdot R^{\alpha_R}_{geo}\cdot \kappa^{\alpha_\kappa}_a\cdot \epsilon^{\alpha_\epsilon}\cdot M^{\alpha_M}_{eff}
$$

In [10]:
def predict_TauE(Mprms, Dprms):
    tau = (Mprms['const'] * (Dprms["IP"]**Mprms["IP"]) * (Dprms["BT"]**Mprms["BT"]) *
          (Dprms["NEL"]**Mprms["NEL"]) * (Dprms["PLTH"]**Mprms["PLTH"]) * (Dprms["RGEO"]**Mprms["RGEO"]) *
          (Dprms["KAREA"]**Mprms["KAREA"]) * (Dprms["EPS"]**Mprms["EPS"]) * (Dprms["MEFF"]**Mprms["MEFF"])
          )
    return tau

In [11]:
predict_TauE(DB2prms, ITER)

10.353061437233006

In [12]:
predict_TauE(DB5prms, ITER)

2.0250652668660973

In [13]:
predict_TauE(DB5prms, DEMO)

5.462009737878593

In [14]:
predict_TauE(DB2prms, DEMO)

33.230622092876644

In [15]:
predict_TauE(DDsprms, ITER)

0.47509334787359064

In [16]:
predict_TauE(UDsprms, ITER)

11.294043186126423

In [17]:
predict_TauE(DDsprms, DEMO)

1.0473879890614215

In [18]:
predict_TauE(UDsprms, DEMO)

34.478956423240135

In [21]:
DB5[["Q95"]].describe()

,Q95
count,6237.000000
mean,3.966204
std,1.148390
min,1.873000
25%,3.232000
50%,3.678000
75%,4.373000
max,11.310000


In [26]:
DB5[DB5.TOK.isin(['MAST'])].DATE.astype(str).apply(lambda x: x[:4]).unique()

array(['2002', '2003', '2004', '2005'], dtype=object)